In [67]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
import os
import json
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [181]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):
    print(game_states[0].game_state[-1]['summary'].keys())
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    social_behavior = np.array([g.player_social_behaviour for g in game_states])
    social_behavior = social_behavior.reshape(-1, 2)
    
    df = np.concatenate((game_name, models, outcomes, resources_delta, social_behavior), axis=1)
    df = pd.DataFrame(df, columns=['game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2', "social_1", "social_2"])
    # for column in ['outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2']:
    #     print(df[column])
    #     df[column] = pd.to_numeric(df[column])
    

    return df
      


In [182]:
BEHAVES = {"": "S", "You are an hostile agent, be cunning and sly." : "H",
         "You should fake being very desperate and in need of resources. Cry and beg to get more resources." : "B",
          "You are a condescending and impatient agent. Dominate your opponents, tell them mean words and how pitiful and invaluable they are, make them obey you." : "C"}






In [271]:
# log_dir_bs = "/Users/patrickjohnchia/Desktop/Projects/negotiation/.logs/buysell"
log_dir = "../experiments/.logs/trading/"
game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
df = compute_game_summary(game_states)




THERE ARE 28 log files
26 Loaded Successfully
dict_keys(['player_goals', 'initial_resources', 'proposed_trade', 'final_response', 'final_resources', 'player_outcome'])


In [272]:
#df["social_1"] = df["social_1"].replace(BEHAVES)
df["social_2"] = df["social_2"].replace(BEHAVES)

In [273]:
social_games = df # df[(df["social_1"].isin(BEHAVES.values())) | (df["social_2"].isin(BEHAVES.values()))].copy()

In [274]:
social_games["model_1"] = social_games["model_1"] + " " + social_games["social_1"]
social_games["model_2"] = social_games["model_2"] + " " + social_games["social_2"]

In [275]:
from elopy import *
def compute_elo(dataf):
    
    i = Implementation()

    for mi in set(dataf["model_1"].tolist() + dataf["model_2"].tolist()):
        i.addPlayer(mi)

    for index, row in dataf.iterrows():
        if row["resource_delta_1"] > row["resource_delta_2"]:
            i.recordMatch(row["model_1"], row["model_2"], winner=row["model_1"])
        elif row["resource_delta_1"] < row["resource_delta_2"]:
            i.recordMatch(row["model_1"], row["model_2"], winner=row["model_2"])
        else:
            i.recordMatch(row["model_1"], row["model_2"], draw=True)
            
    for ratings in sorted(i.getRatingList(), key=lambda x: x[1], reverse=True):
        print(ratings[0], round(ratings[1], 2))

In [276]:
compute_elo(social_games.sample(frac=1))

gpt-4-1106-preview H 1104.03
gpt-4-1106-preview B 1026.04
gpt-4-1106-preview  956.5
gpt-4-1106-preview S 913.43


In [277]:
from collections import Counter

Counter([a+" VS " + b for a,b in social_games[["model_1", "model_2"]].values])

Counter({'gpt-4-1106-preview  VS gpt-4-1106-preview S': 10,
         'gpt-4-1106-preview  VS gpt-4-1106-preview H': 9,
         'gpt-4-1106-preview  VS gpt-4-1106-preview B': 7})

In [278]:
social_games

,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2,social_1,social_2
0,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
1,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: -5.0, Y: 2.0","X: 5.0, Y: -2.0",-3.0,3.0,,S
2,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
3,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,S
4,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview S,"X: -5.0, Y: 8.0","X: 5.0, Y: -8.0",3.0,-3.0,,S
5,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: -2.0, Y: 1.0","X: 2.0, Y: -1.0",-1.0,1.0,,H
6,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: -10.0, Y: 7.0","X: 10.0, Y: -7.0",-3.0,3.0,,H
7,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
8,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
9,TradingGame,gpt-4-1106-preview,gpt-4-1106-preview H,"X: 0, Y: 0","X: 0, Y: 0",0.0,0.0,,H
